In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime
# import db_model
import utils
import argparse
import naver_crawler
import requests
import pandas as pd
from selenium.common.exceptions import TimeoutException

In [3]:
def get_post_info(keyword: str, start_date: str, end_date: str, num: int) :

    post_list = []
    comment_list = []
    

    try:
        first_url = f"https://section.blog.naver.com/Search/Post.nhn?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=" \
                    f"{start_date}&endDate={end_date}&keyword={keyword}"
        print(first_url)
        chrome_options = wd.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = wd.Chrome(executable_path="D:/chromedriver.exe",
                           chrome_options=chrome_options)
        driver.get(first_url)
        time.sleep(1.5)  # 안하면 못불러옴

        gunsu = driver.find_element_by_css_selector(
            '#content > section > div.category_search > div.search_information > span > span > em').text
        print(f'{keyword}에 대한 검색 결과 입니다. {gunsu}')


        # 검색 게시글 수 입력 -> 개수 맞게 페이지수 넘김 -> 해당 포스팅 링크 수집
        if num <= 7:
            loop_num = 1
        else:
            loop_num = num//7+1

        for i in range(loop_num):

            loop_url = f"https://section.blog.naver.com/Search/" \
                       f"Post.nhn?pageNo={i + 1}&rangeType=PERIOD&orderBy=sim&startDate=" \
                       f"{start_date}&endDate={end_date}&keyword={keyword}"

            driver.get(loop_url)
            time.sleep(3)
            elems = driver.find_elements_by_css_selector("a.desc_inner")
            post_urls = [elem.get_attribute('href') for elem in elems]

            for j, jj in enumerate(post_urls):
                print(jj)
                blogno = jj.split("/")[-1]
                blogid = re.compile(".blog.me").sub('', jj.split("/")[-2])
                post_url = f"https://blog.naver.com/PostView.nhn?blogId={blogid}&logNo={blogno}" \
                           f"&redirect=Dlog&widgetTypeCall=true&directAccess=false"
                driver.get(post_url)
                time.sleep(5)

                # 좋아요 기능이 없고 댓글만 있는 경우는 통과
                try:
                    comment_click = driver.find_elements_by_css_selector(
                        '#printPost1 > tbody > tr > td.bcc > div.post-btn.post_btn2 >'
                        ' div.wrap_postcomment > div > a')[1]
                except IndexError:
                    continue

                comment_click.click()
                time.sleep(5)
                html_source = driver.page_source
                soup = BeautifulSoup(html_source, 'lxml')

                follow_res = requests.get("https://section.blog.naver.com/connect/ViewMoreFollowers.nhn",
                                          params={"blogId": blogid})
                time.sleep(2)
                follow_soup = BeautifulSoup(follow_res.content, 'lxml')

                # 블로그 종류 다른 경우
                try:
                    title = soup.find("div", {"class": "se-module se-module-text se-title-text"}).get_text(strip=True)
                except AttributeError:
                    try:
                        title = soup.find("span", {"class": "pcol1 itemSubjectBoldfont"}).get_text(strip=True)
                    except AttributeError:
                        title = soup.find("h3", {"class": "se_textarea"}).get_text(strip=True)
                # 블로그 종류 다른 경우
                try:
                    user_name = soup.find("span", {"class": "nick"}).find("a").text
                except AttributeError:
                    try:
                        user_name = soup.find("strong", {"id": "nickNameArea"}).text
                    except AttributeError:
                        user_name = soup.find("a", {"class": "link pcol2"}).text

                # 블로그 종류 다른 경우
                try:
                    posting_date = utils.conv_date_naver(
                        soup.find("span", {"class": "se_publishDate pcol2"}).text)
                except AttributeError:
                    try:
                        posting_date = utils.conv_date_naver(
                            soup.find("p", {"class": "date fil5 pcol2 _postAddDate"}).text)
                    except AttributeError:
                        posting_date = utils.conv_date_naver(
                            soup.find("span", {"class": "se_publishDate pcol2"}).text)

                # 블로그 종류 다른 경우
                try:
                    contents = re.compile("\\u200b").sub(' ', soup.find("div", {"class": "se-main-container"}).get_text(
                        strip=True))
                except AttributeError:
                    try:
                        contents = re.compile("\\u200b").sub(' ', soup.find("div", {"id": "postViewArea"}).get_text(
                            strip=True))
                    except AttributeError:
                        contents = re.compile("\\u200b").sub(' ', soup.find(
                            "div", {"class": "se_component_wrap sect_dsc __se_component_area"}).get_text(strip=True))
                # 값 없는 경우
                try:
                    like_count = int(soup.find("em", {"class": "u_cnt _count"}).text)
                except ValueError:
                    like_count = 0

                post_dict = {
                    'unique_id': blogno,
                    'keyword': keyword,
                    'title': utils.addslashes(title),
                    'user_id': blogid,
                    'user_name': user_name,
                    'posting_date': posting_date,
                    'view_count': 0,
                    'like_count': like_count,
                    'dislike_count': 0,
                    'contents': utils.addslashes(contents),
                    'user_follow': 0,
                    # 'user_follower': user_follower,
                    'user_medias': int(
                        soup.select("#category-list > div > ul > li.allview > span")[0].text.replace("(", '').replace(
                            ")", '')),
                    # 'comment_count': comment_count
                }
                time.sleep(1)
                # 쿼리
                # if self.args.is_db:
                #     body_is_new = self.db_model.set_data_body(3, post_dict)

                post_list.append(post_dict)

                for k in range(len(soup.find_all("span", {"class": "u_cbox_nick"}))):
                    comment_dict = {
                        "unique_id": blogno,
                        "keyword": keyword,
                        "user_name": soup.find_all("span", {"class": "u_cbox_nick"})[k].text,
                        "comment_date": datetime.strptime(soup.find_all("span", {"class": "u_cbox_date"})[k].text,
                                                          '%Y.%m.%d. %H:%M').strftime("%Y-%m-%d %H:%M:%S"),
                        "comment": utils.addslashes(
                            soup.find_all("span", {"class": "u_cbox_contents"})[k].text),
                        "comment_like": 0
                    }
                    time.sleep(1)


                    comment_list.append(comment_dict)

                comm_page_path = driver.find_elements_by_class_name("u_cbox_page")

                for k in range(len(comm_page_path) - 1):

                    comm_page_path = driver.find_elements_by_class_name("u_cbox_page")
                    comm_page_path[k].click()
                    time.sleep(1)
                    html_source = driver.page_source
                    soup = BeautifulSoup(html_source, 'lxml')

                    for l in range(len(soup.find_all("span", {"class": "u_cbox_nick"}))):
                        comment_dict = {
                            "unique_id": blogno,
                            "keyword": keyword,
                            "user_name": soup.find_all("span", {"class": "u_cbox_nick"})[l].text,
                            "comment_date": datetime.strptime(soup.find_all("span", {"class": "u_cbox_date"})[l].text,
                                                              '%Y.%m.%d. %H:%M').strftime("%Y-%m-%d %H:%M:%S"),
                            "comment": utils.addslashes(
                                soup.find_all("span", {"class": "u_cbox_contents"})[l].text),
                            "comment_like": 0
                        }
                        time.sleep(1)

                        comment_list.append(comment_dict)

    except TimeoutException as ex:
        isrunning = 0
        # print("Exception has been thrown. " + str(ex)) # 출력 부분 
        Driver.close()
                        
    return post_list, comment_list

# num = int(input("몇 페이지 까지 크롤링 할까요? \n => "))
# keyword = input("검색하실 키워드를 입력해주세요 \n =>")
# start_date = input("검색 시작일을 입력해주세요 ex) 2000-01-01 \n =>")
# end_date = input("검색 종료일을 입력해주세요 ex) 2000-01-01 \n =>")

post_list, comment_list = get_post_info(keyword = '산업공학과', start_date = '2019-01-13', end_date='2022-01-13', num=800)

# 게시글 
post_list = pd.DataFrame(post_list)
post_list

,unique_id,keyword,title,user_id,user_name,posting_date,view_count,like_count,dislike_count,contents,user_follow,user_medias
0,222269307925,산업공학과,산업공학과 선택과목 고르는게 어렵다면?,beaguen,입시랩,2021-03-09 11:31:00,0,20,0,많은 학생들이 궁금해하는 정보가바로 대학 입시에 대한 정보일 것 같아요 수시 등급도...,0,1052
1,222582265319,산업공학과,"잘나가는 CEO, 다 산업공학과 출신이네!",pjhkbj,공부하는 에디터,2021-12-02 00:00:00,0,11,0,최근 재계에서 재밌는 공통점이 발견되었는데요.바로 산업공학과 출신 CEO(최고경영자...,0,202
2,222568787426,산업공학과,"학점은행제 산업공학과,중퇴자도 빠르게 학사학위 취득가능!",kimyouls,학습플래너,2021-11-15 15:28:00,0,3,0,"안녕하세요,여러분 모두들 좋은 주말 보내셨나요?! 제 주변 지인들은 날씨가 더 추...",0,122
3,222612519793,산업공학과,"신은 마음을, 사람은 겉모습을 본다지만 / 박희준 연세대 산업공학과 교수",starkujk,하모니,2022-01-04 11:52:00,0,3,0,"아침을 열며신은 마음을, 사람은 겉모습을 본다지만/ 박희준 연세대 산업공학과 교수입...",0,6088
4,222501567496,산업공학과,"취업률 80% 이상 달성, 지역 산업 맞춤형 전문 인력을 양성하는 산업공학과를 소개...",discoveryj,아임수퍼스타,2021-09-10 16:21:00,0,11,0,▲전주대학교 산업공학과오늘은 전주대학교 \'산업공학과\'를 소개해보는 시간을 가져보...,0,1720
...,...,...,...,...,...,...,...,...,...,...,...,...
646,222501637733,산업공학과,컴퓨터공학과 수시모집 드디어 시작,woosukuni,우석소식,2021-09-10 18:00:00,0,4,0,#컴퓨터공학과#우석대학교#재학생후기요즘엔 초등학생 때부터 정규 교과과정에코딩 수업이...,0,1274
647,222561642771,산업공학과,[조선해양공학과] 이제명 교수,pnupurm,푸르미,2021-11-08 09:14:00,0,0,0,국내 최초 수소차 액체수소 연료탱크 개발 주도…산자부 소재부품개발사업 선정조선해양공...,0,2081
648,222492780831,산업공학과,금형설계 및 금형공학과 관심있다면!,yuhancollege,유한대학교,2021-09-02 17:23:00,0,7,0,안녕하세요. MC유한입니다!일상생활 중에 사용하는 여러가지도구 및 개체들은 필요에 ...,0,2940
649,222317679223,산업공학과,건국대 줄기세포재생공학과 무조건 성공할 거라는 마인드로 김유정 멘토,opensky_edu,오픈스카이,2021-04-21 11:26:00,0,12,0,안녕하세요! 오픈스카이입니다. \'줄기세포\'라는단어들어본 적이 있으실까요? 아마의...,0,644


In [4]:
# 댓글 
comment_list = pd.DataFrame(comment_list)
comment_list

,unique_id,keyword,user_name,comment_date,comment,comment_like
0,222269307925,산업공학과,sunnyfactorystory,2021-03-09 19:18:00,포스팅 잘 보고 갑니다. 좋은 글 감사해요~\n,0
1,222269307925,산업공학과,멍뭉이,2021-03-10 10:00:00,글보고 공감 누르고 갑니다 : ),0
2,222269307925,산업공학과,줄리엣러브,2021-03-11 12:26:00,쉬는 날은 빨리도 가요. 시간이 아깝네요. 잇님..^^ 우리집 아이들은 나가자고 아...,0
3,222269307925,산업공학과,신쓰양,2021-03-11 12:41:00,수고한 당신 떠나라고 했나요. 저는 지난 주말에 남편이 특별 이벤트를 준비해 줬어요...,0
4,222269307925,산업공학과,엘라인필라테스 고잔,2021-03-13 12:13:00,이웃님. 휴일 잘 지내셨나요? 또 새로운 한주가 시작했네요. 미세먼지가 많아서 아쉽...,0
...,...,...,...,...,...,...
644,222501691495,산업공학과,팀공조시스템,2021-09-13 10:36:00,블로그 글 잘 봤습니다답방 부탁 드립니다월요일 오늘도 힘내세요!,0
645,221549928196,산업공학과,횬스타일,2019-05-31 13:55:00,요즘 과가 넘 많아서 뭐가뭔지 모르겠음 신청해서 자세히 알아보면 좋겠네,0
646,222556119813,산업공학과,어스로하,2021-11-02 15:32:00,잘 보고 갑니다ㅎㅎㅎ,0
647,222501637733,산업공학과,자녀 폰관리 스카이홈,2021-09-11 21:06:00,아들 서울대 보내려고 스마트폰 시간제한 앱을 직접 만든 개발자 아빠입니다강남 엄마들...,0


In [5]:
post_list.to_csv("post_list.csv", encoding='utf-8-sig')

In [6]:
comment_list.to_csv("comment_list.csv", encoding='utf-8-sig')